<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/BERT_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORTANT

Open/modify any ipynb file only with the editor in which it was created, don't open a JupyterLab ipynb with Colab and vice versa, don't open any ipynb with an editor that can render notebook display, like VS Code or VSCodium.

---

Do not use conda in Colab notebook, do not try to install with conda, use pip.



In [1]:
!pip install -q accelerate
!pip install -q huggingface_hub
!pip install -q scikit-learn
!pip install -q transformers datasets
!pip install -q wandb

import json
import numpy as np
import os
import sys
import time
import torch
import wandb

from datasets              import DatasetDict
from google.colab          import auth, drive, files, userdata
from huggingface_hub       import create_repo, login, upload_file
from huggingface_hub.utils import RepositoryNotFoundError
from sklearn.metrics       import accuracy_score, average_precision_score, classification_report, f1_score, precision_score, recall_score, roc_auc_score
from torch.utils.data      import DataLoader
from tqdm.auto             import tqdm
from transformers          import AutoModelForSequenceClassification, AutoTokenizer, EvalPrediction, Trainer, TrainingArguments


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
"""
# Check the Python version
print(sys.version)
print()

# Get the installed packages (you can see that conda is not installed (do not install it))
!pip list
print()

# Check system information
!cat /etc/os-release
!uname -m
print()

# Check the GPU details (only if the runtime type is T4 GPU)
#!nvidia-smi
#print()

# Check RAM
!free -h
print()

# Check disk space
!df -h
print()

# Get environment variables
for key, value in os.environ.items():
    print(f"{key}: {value}")
"""
!python -V

print(f"currentdir: {os.getcwd()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

datasetDict_zip_file_name = "dataset_11_1000.zip"
datasetDict_dir_name      = os.path.splitext(datasetDict_zip_file_name)[0]
print(f"datasetDict_zip_file_name: {datasetDict_zip_file_name}")
print(f"datasetDict_dir_name     : {datasetDict_dir_name}")
print()

batch_size    = 8
epochs        = 5
learning_rate = 2e-5
threshold     = 0.2

run_name = f"BERT-multilabel-{datasetDict_dir_name}-batch{batch_size}-epochs{epochs}-lr{learning_rate}-threshold{threshold}"
print(f"run_name                 : {run_name}")

Python 3.10.12
currentdir: /content
device: cuda
datasetDict_zip_file_name: dataset_11_1000.zip
datasetDict_dir_name     : dataset_11_1000

run_name                 : BERT-multilabel-dataset_11_1000-batch8-epochs5-lr2e-05-threshold0.2


## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [3]:
def upload_unzip_dataset(file_name=datasetDict_zip_file_name):
  # Check if the file exists
  if not os.path.exists(file_name):
    print(f"'{file_name}' not found in /content. Uploading...")
    uploaded_files = files.upload()                              # Prompt file upload dialog
    if file_name not in uploaded_files:
      raise FileNotFoundError(f"'{file_name}' was not uploaded. Please try again.")
    print(f"'{file_name}' successfully uploaded to /content")
    uploaded_file_name = list(uploaded_files.keys())[0]          # Get the name of the uploaded file

    !unzip {uploaded_file_name}

    unzipped_dir_name = os.path.splitext(uploaded_file_name)[0]
    assert unzipped_dir_name==datasetDict_dir_name, "unzipped_dir_name != datasetDict_dir_name"
  else:
    print(f"'{datasetDict_dir_name}' already exists in /content.")


In [4]:
upload_unzip_dataset(datasetDict_zip_file_name)

'dataset_11_1000.zip' not found in /content. Uploading...


Saving dataset_11_1000.zip to dataset_11_1000.zip
'dataset_11_1000.zip' successfully uploaded to /content
Archive:  dataset_11_1000.zip
  inflating: dataset_11_1000/dataset_dict.json  
  inflating: dataset_11_1000/test/data-00000-of-00001.arrow  
  inflating: dataset_11_1000/test/dataset_info.json  
  inflating: dataset_11_1000/test/state.json  
  inflating: dataset_11_1000/train/data-00000-of-00001.arrow  
  inflating: dataset_11_1000/train/dataset_info.json  
  inflating: dataset_11_1000/train/state.json  
  inflating: dataset_11_1000/validation/data-00000-of-00001.arrow  
  inflating: dataset_11_1000/validation/dataset_info.json  
  inflating: dataset_11_1000/validation/state.json  


In [5]:
# Hugging Face Authenticate

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')
login(token=hf_token)

# Verify
!huggingface-cli whoami


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


claudelepere


In [6]:
# Create the skill_classification repo on the Hugging Face Hub

HF_name         = "claudelepere/skill_classification"
repo_id_model   = HF_name
repo_id_dataset = HF_name

repo_model_url = create_repo(
    repo_id   = repo_id_model,
    repo_type = "model",
    private   = True,
    exist_ok  = True
    )
print(f"Repo model url: {repo_model_url} created successfully as a private repo.")

repo_dataset_url = create_repo(
    repo_id   = repo_id_dataset,
    repo_type = "dataset",
    private   = True,
    exist_ok  = True
    )
print(f"Repo datasets url: {repo_dataset_url} created successfully as a private repo.")

repo_id_dataset = f"datasets/{HF_name}"

print(f"repo_id_model: {repo_id_model}")
print(f"repo_id_dataset: {repo_id_dataset}")


Repo model url: https://huggingface.co/claudelepere/skill_classification created successfully as a private repo.
Repo datasets url: https://huggingface.co/datasets/claudelepere/skill_classification created successfully as a private repo.
repo_id_model: claudelepere/skill_classification
repo_id_dataset: datasets/claudelepere/skill_classification


In [7]:
# W&B initialization

os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ
wandb_api_key               = os.environ.get('WANDB_API_KEY')
wandb.login(key=wandb_api_key)

try:
  wandb.init(
      project = "skill_classification",
      name    = run_name,
      entity  = "claudelepere-c-cile-cy",
      config  = {
          "learning_rate": 2e-5,
          "epochs"       : 5,
          "batch_size"   : 8
          }
      )
except wandb.errors.CommError as err:
  print(f"CommError: {err}")
except Exception as exc:
  print(f"Exception: {exc}")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: claudelepere (claudelepere-c-cile-cy). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# My fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a **linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels)**, indicating the unnormalized scores for a number of labels for every example in the batch.


In [8]:
# Create the dataset: 3 Hugging Face Dataset in a Hugging Face DatasetDict

datasetDict = DatasetDict.load_from_disk(datasetDict_dir_name)

print(f"datasetDict: {type(datasetDict)} {datasetDict.shape}\n{datasetDict}")


datasetDict: <class 'datasets.dataset_dict.DatasetDict'> {'train': (700, 8), 'validation': (150, 8), 'test': (150, 8)}
DatasetDict({
    train: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 700
    })
    validation: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 150
    })
    test: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 150
    })
})


As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

Let's test the first example of the training split:

In [9]:
example = datasetDict['train'][0]
print(f"example: {type(example)} {example.keys()}\n{example}")

example: <class 'dict'> dict_keys(['id', 'text', '390', '135', '136', '137', '138', '139'])
{'id': 307149, 'text': "Umicore - Senior Network Platform engineer BGP,OSPF,VLAN, routers, switches Umicore About Umicore Reducing harmful vehicle emissions. Giving new life to used metals. Powering the cars of the future. As a global materials and technology group, we apply our specialist knowledge to offer materials and solutions that are essential to everyday life. We aim to be a clear world leader in materials for clean mobility and recycling and have turned our sustainability approach into an even greater competitive advantage. With ambitions like this, imagine what you could do? About our support services (Information Systems) Umicore's commitment to creating a sustainable future relies on innovation in all areas of business. As we keep pushing our boundaries and strive to set new industry standards, embracing advanced technologies has never been more essential. Digital technologies are co

In [10]:
# Create the label list and the id2label and label2id mappings.

"""
dataset 7_1000_125_125  ,  48 labels
dataset 7_128_18_54     ,  42 labels
dataset 8910_1087_68_204, 206 labels
dataset 11_1000         ,   6 labels
"""

labels = [label for label in datasetDict['train'].features.keys() if label not in ['id', 'text']]
labels.sort()
print(f"labels: {type(labels)} {len(labels)}\n{labels}")

id2label = {idx:label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

label2id = {label:idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

labels: <class 'list'> 6
['135', '136', '137', '138', '139', '390']
id2label: <class 'dict'> 6
{0: '135', 1: '136', 2: '137', 3: '138', 4: '139', 5: '390'}
label2id: <class 'dict'> 6
{'135': 0, '136': 1, '137': 2, '138': 3, '139': 4, '390': 5}


The dataset consists of texts, labeled with one or more skills.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [11]:
# Upload to the HF repo_id_dataset the label list as a JSON file

labels_path = "labels.json"
with open(labels_path, 'w') as f:
    json.dump(labels, f)
print(f"labels saved to {labels_path}")

upload_file(
    path_or_fileobj = labels_path,
    path_in_repo    = labels_path,
    repo_id         = HF_name,
    repo_type       = "dataset"
    )
print(f"labels uploaded to https://huggingface.co/datasets/{HF_name}/tree/main/{labels_path}")

labels saved to labels.json


No files have been modified since last commit. Skipping to prevent empty commit.


labels uploaded to https://huggingface.co/datasets/claudelepere/skill_classification/tree/main/labels.json


## Preprocess data

As models like BERT don't expect text as direct input, but rather **`input_ids`**, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a **matrix of shape (batch_size, num_labels)**. Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' **BCEWithLogitsLoss** (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

### Preprocess
note: examples, not example, because batched=True => examples is a batch

In [12]:
# Tokenize 'text' in the 3 datasets, train, validation and test

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
def preprocess_data(examples, indices):
  text = examples['text']    # Batch of texts

  encoding = tokenizer(
      text,                           # Tokenize text
      truncation     = True,
      padding        = 'max_length',
      max_length     = 512,
      return_tensors = 'pt'           # Return PyTorch tensors
      )

  # Create an empty label matrix
  labels_matrix = torch.zeros((len(text), len(labels)), dtype=torch.float32)
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  # Populate label matrix
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")
    if label in examples:
      labels_matrix[:, idx] = torch.tensor(
          [1.0 if val else 0.0 for val in examples[label]],
          dtype=torch.float32
          )
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  # Add labels to the encoding
  encoding['labels'] = labels_matrix
  #print(f"encoding['labels']: {encoding['labels']}")

  return encoding

### Tokens are not words, so the text is truncated

Tokens are the units the tokenizer produces after processing the text. In BERT (and most Transformer models), tokenization uses **subword units** (from methods like WordPiece or Byte-Pair Encoding). A single word can be split into multiple tokens if it's not in the model's vocabulary.

Example: 'unbelievable' => 3 tokens, 'un', '##believe', '##able'


In [14]:
def get_truncated_part(text):
  tokens = tokenizer(
      text,
      truncation                = True,
      padding                   = 'max_length',
      max_length                = 512,
      return_overflowing_tokens = True,
      return_tensors            = None
      )
  print(f"tokens.keys(): {tokens.keys()}")

  # Get the truncated tokens
  truncated_ids = tokens["input_ids"][0]
  print(f"truncated_ids: {type(truncated_ids)} {truncated_ids}")
  #overflow_ids  = tokens["overflow_to_sample_mapping"][0]
  #print(f"overflow_ids: {type(overflow_ids)} {overflow_ids}")

  # Decode the tokens back to text
  truncated_text = tokenizer.decode(truncated_ids, skip_special_tokens=True)
  #overflow_text  = tokenizer.decode(overflow_ids, skip_special_tokens=True)

  print(f"original_text :\n{text}")
  print(f"truncated_text:\n{truncated_text}")
  #print(f"overflow_text:\n{overflow_text}")

  original_tokens  = tokenizer.tokenize(text)
  truncated_tokens = tokenizer.tokenize(truncated_text)
  #overflow_tokens  = tokenizer.tokenize(overflow_text)

  print(f"original_tokens count : {len(original_tokens)}")
  print(f"truncated_tokens count: {len(truncated_tokens)}")
  #print(f"overflow_tokens count: {len(overflow_tokens)}")


In [15]:
example_text = datasetDict['train'][0]['text']
get_truncated_part(example_text)


Token indices sequence length is longer than the specified maximum sequence length for this model (835 > 512). Running this sequence through the model will result in indexing errors


tokens.keys(): dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'])
truncated_ids: <class 'list'> [101, 8529, 11261, 2890, 1011, 3026, 2897, 4132, 3992, 1038, 21600, 1010, 9808, 14376, 1010, 1058, 5802, 1010, 2799, 2869, 1010, 15924, 8529, 11261, 2890, 2055, 8529, 11261, 2890, 8161, 17631, 4316, 11768, 1012, 3228, 2047, 2166, 2000, 2109, 11970, 1012, 2373, 2075, 1996, 3765, 1997, 1996, 2925, 1012, 2004, 1037, 3795, 4475, 1998, 2974, 2177, 1010, 2057, 6611, 2256, 8325, 3716, 2000, 3749, 4475, 1998, 7300, 2008, 2024, 6827, 2000, 10126, 2166, 1012, 2057, 6614, 2000, 2022, 1037, 3154, 2088, 3003, 1999, 4475, 2005, 4550, 12969, 1998, 17874, 1998, 2031, 2357, 2256, 15169, 3921, 2046, 2019, 2130, 3618, 6975, 5056, 1012, 2007, 19509, 2066, 2023, 1010, 5674, 2054, 2017, 2071, 2079, 1029, 2055, 2256, 2490, 2578, 1006, 2592, 3001, 1007, 8529, 11261, 2890, 1005, 1055, 8426, 2000, 4526, 1037, 9084, 2925, 16803, 2006, 8144, 1999, 2035, 2752, 1997, 2449, 1012, 20

In [16]:
# Create the 3 encoded datasets, train, validation and test

encoded_dataset = datasetDict.map(
    preprocess_data,
    batched        = True,
    remove_columns = datasetDict['train'].column_names,
    with_indices   = True
    )
train_dataset      = encoded_dataset['train']
validation_dataset = encoded_dataset['validation']
test_dataset       = encoded_dataset['test']
print(f"encoded_dataset: {type(encoded_dataset)} {encoded_dataset.shape}\n{encoded_dataset}")
print(f"train_dataset: {type(train_dataset)} {train_dataset.shape}")
print(f"validation_dataset: {type(validation_dataset)} {validation_dataset.shape}")
print(f"test_dataset: {type(test_dataset)} {test_dataset.shape}")

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

encoded_dataset: <class 'datasets.dataset_dict.DatasetDict'> {'train': (700, 4), 'validation': (150, 4), 'test': (150, 4)}
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 700
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
})
train_dataset: <class 'datasets.arrow_dataset.Dataset'> (700, 4)
validation_dataset: <class 'datasets.arrow_dataset.Dataset'> (150, 4)
test_dataset: <class 'datasets.arrow_dataset.Dataset'> (150, 4)


In [17]:
example = encoded_dataset['train'][0]

print(f"example: {type(example)} {example.keys()}\n{example}")
print()
print(f"example['input_ids']: {type(example['input_ids'])} {len(example['input_ids'])}\n{example['input_ids']}")
print(f"example['token_type_ids']: {type(example['token_type_ids'])} {len(example['token_type_ids'])}\n{example['token_type_ids']}")
print(f"example['attention_mask']: {type(example['attention_mask'])} {len(example['attention_mask'])}\n{example['attention_mask']}")
print(f"example['labels']:  {type(example['labels'])} {len(example['labels'])}\n{example['labels']}")

example: <class 'dict'> dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
{'input_ids': [101, 8529, 11261, 2890, 1011, 3026, 2897, 4132, 3992, 1038, 21600, 1010, 9808, 14376, 1010, 1058, 5802, 1010, 2799, 2869, 1010, 15924, 8529, 11261, 2890, 2055, 8529, 11261, 2890, 8161, 17631, 4316, 11768, 1012, 3228, 2047, 2166, 2000, 2109, 11970, 1012, 2373, 2075, 1996, 3765, 1997, 1996, 2925, 1012, 2004, 1037, 3795, 4475, 1998, 2974, 2177, 1010, 2057, 6611, 2256, 8325, 3716, 2000, 3749, 4475, 1998, 7300, 2008, 2024, 6827, 2000, 10126, 2166, 1012, 2057, 6614, 2000, 2022, 1037, 3154, 2088, 3003, 1999, 4475, 2005, 4550, 12969, 1998, 17874, 1998, 2031, 2357, 2256, 15169, 3921, 2046, 2019, 2130, 3618, 6975, 5056, 1012, 2007, 19509, 2066, 2023, 1010, 5674, 2054, 2017, 2071, 2079, 1029, 2055, 2256, 2490, 2578, 1006, 2592, 3001, 1007, 8529, 11261, 2890, 1005, 1055, 8426, 2000, 4526, 1037, 9084, 2925, 16803, 2006, 8144, 1999, 2035, 2752, 1997, 2449, 1012, 2004, 2057, 2562, 6183, 2256,

In [18]:
tokenizer.decode(example['input_ids'])

"[CLS] umicore - senior network platform engineer bgp, ospf, vlan, routers, switches umicore about umicore reducing harmful vehicle emissions. giving new life to used metals. powering the cars of the future. as a global materials and technology group, we apply our specialist knowledge to offer materials and solutions that are essential to everyday life. we aim to be a clear world leader in materials for clean mobility and recycling and have turned our sustainability approach into an even greater competitive advantage. with ambitions like this, imagine what you could do? about our support services ( information systems ) umicore ' s commitment to creating a sustainable future relies on innovation in all areas of business. as we keep pushing our boundaries and strive to set new industry standards, embracing advanced technologies has never been more essential. digital technologies are constantly evolving and present new opportunities for us. as next generation corporate it department, we 

In [19]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['138', '139']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [20]:
# Set PyTorch format to ensures correctness and compatibility with PyTorch pipelines

# The 3 Hugging Face Dataset are formatted as PyTorch Dataset
encoded_dataset.set_format('torch')

## Define the model

Here we define a **model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top**. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [**BCEWithLogitsLoss**](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [21]:
# Define the model

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
    )

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train the model

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [22]:
batch_size  = batch_size
metric_name = "f1"

### TrainingArguments

In [23]:
output_dir = "training_results"  # where model predictions and checkpoints will be written during training
args = TrainingArguments(
    output_dir                  = output_dir,
    overwrite_output_dir        = True,
    logging_dir                 = "logs",
    logging_steps               = 50,
    save_steps                  = 500,
    save_total_limit            = 2,
    eval_strategy               = "epoch",
    save_strategy               = "epoch",
    learning_rate               = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    num_train_epochs            = epochs,
    weight_decay                = 0.01,
    load_best_model_at_end      = True,
    metric_for_best_model       = metric_name,
    run_name                   = run_name,
    report_to                  = "wandb"
    )

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [24]:
# Metrics
#   source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

def multi_label_metrics(predictions, labels):
    average = 'micro'    # 'micro' or 'weighted'

    # first, apply sigmoid on predictions whose shape is (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs   = sigmoid(torch.Tensor(predictions))

    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # finally, compute metrics
    y_true               = labels
    f1                   = f1_score               (y_true=y_true, y_pred=y_pred, average=average)    #, zero_division=1)
    precision            = precision_score        (y_true=y_true, y_pred=y_pred, average=average)    #, zero_division=1)
    recall               = recall_score           (y_true=y_true, y_pred=y_pred, average=average)    #, zero_division=1)
    roc_auc              = roc_auc_score          (y_true=y_true, y_score=probs, average=average)
    precision_recall_auc = average_precision_score(y_true=y_true, y_score=probs, average=average)
    accuracy             = accuracy_score         (y_true=y_true, y_pred=y_pred)

    # return as dictionary
    metrics = {
        'f1'                  : f1,
        'precision'           : precision,
        'recall'              : recall,
        'roc_auc'             : roc_auc,
        'precision_recall_auc': precision_recall_auc,
        'accuracy'            : accuracy
        }

    return metrics

In [25]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(
        predictions = preds,
        labels      = p.label_ids
        )
    return result

Let's verify a batch as well as a forward pass:

In [26]:
print(f"inputids:        {type(encoded_dataset['train']['input_ids'][0])}\t{encoded_dataset['train']['input_ids'][0].shape}")
print(f"token_type_ids': {type(encoded_dataset['train']['token_type_ids'][0])}\t{encoded_dataset['train']['token_type_ids'][0].shape}")
print(f"attention_mask:  {type(encoded_dataset['train']['attention_mask'][0])}\t{encoded_dataset['train']['attention_mask'][0].shape}")
print(f"labels:          {type(encoded_dataset['train'][0]['labels'])}\t{encoded_dataset['train'][0]['labels'].shape}")

inputids:        <class 'torch.Tensor'>	torch.Size([512])
token_type_ids': <class 'torch.Tensor'>	torch.Size([512])
attention_mask:  <class 'torch.Tensor'>	torch.Size([512])
labels:          <class 'torch.Tensor'>	torch.Size([6])


In [27]:
# Execute a forward pass for debugging or verificatin purposes (cf. BERT_3_1 in Notion BERT database)

outputs = model(
    input_ids      = encoded_dataset['train']['input_ids'][0].unsqueeze(0),
    attention_mask = encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
    labels         = encoded_dataset['train'][0]['labels'].unsqueeze(0)
    )

print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

outputs: <class 'transformers.modeling_outputs.SequenceClassifierOutput'> odict_keys(['loss', 'logits'])
SequenceClassifierOutput(loss=tensor(0.5930, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0156, -0.1368, -0.8389,  0.3842,  0.2567,  0.1925]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


Let's start training!

In [28]:
# Create the trainer

trainer = Trainer(
    model,
    args,
    train_dataset = encoded_dataset["train"],
    eval_dataset  = encoded_dataset["validation"],
    tokenizer     = tokenizer,
    compute_metrics=compute_metrics
    )


<ipython-input-28-2d6589e12c30>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [29]:
# Train, save the results as a JSON file

train_output  = trainer.train()

train_results = {
    'global_step':   train_output.global_step,    # total steps completed during training
    'training_loss': train_output.training_loss,  # average loss during training
    'metrics':       train_output.metrics         # dictionary of metrics
}

# Save train results
with open("train_results.json", "w") as f:
  json.dump(train_results, f, indent=4)

Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Roc Auc,Precision Recall Auc,Accuracy
1,0.461000,0.404802,0.784713,0.684444,0.919403,0.888860,0.809244,0.233333
2,0.347900,0.390950,0.786346,0.682018,0.928358,0.898275,0.823745,0.233333
3,0.330500,0.388896,0.788875,0.684211,0.931343,0.901957,0.818413,0.226667
4,0.296200,0.368089,0.790875,0.687225,0.931343,0.909449,0.840799,0.253333
5,0.272800,0.367005,0.794904,0.693333,0.931343,0.910342,0.833188,0.273333


In [30]:
print("Training successfully completed.")

Training successfully completed.


## Evaluate

After training, we evaluate our model on the validation set.

In [31]:
def get_results(model, dataset, batch_size, threshold):

  # Set the model to evaluation mode to disable dropout and other training-specific behaviors
  model.eval()

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

  all_preds       = []
  all_probs       = []
  all_true_labels = []

  for batch in tqdm(test_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
      outputs = model(**batch)
    logits = outputs.logits

    # Convert logits to probabilities and probabilities to predictions
    sigmoid = torch.nn.Sigmoid()
    probs   = sigmoid(logits).cpu().numpy()    # Convert to Numpy
    preds   = (probs > threshold).astype(int)  # Convert to binary Numpy array

    # Accumulate probabilities, predictions and labels
    all_probs.append(probs)
    all_preds.append(preds)
    all_true_labels.append(batch['labels'].cpu().numpy())

  # Concatenate results from all batches
  all_probs       = np.concatenate(all_probs, axis=0)        # shape: [num_samples, num_labels]
  all_preds       = np.concatenate(all_preds, axis=0)        # shape: [num_samples, num_labels]
  all_true_labels = np.concatenate(all_true_labels, axis=0)  # shape: [num_samples, num_labels]

  print(f"all_probs:       {type(all_probs)} {all_probs.shape}")
  print(f"all_preds:       {type(all_preds)} {all_preds.shape}")
  print(f"all_true_labels: {type(all_true_labels)} {all_true_labels.shape}")

  # Classification report for precision, recall, F1 score
  print(classification_report(
      y_true        = all_true_labels,
      y_pred        = all_preds,
      target_names  = labels,
      zero_division = 0
      ))

  # ROC AUC for multi-label classification
  roc_auc = roc_auc_score(
      y_true  = all_true_labels,
      y_score = all_probs,
      average = 'micro'
      )
  print(f"ROC AUC: {roc_auc}")

In [32]:
# First evaluate results NO SAVE

get_results(model=model, dataset=validation_dataset, batch_size=batch_size, threshold=threshold)

  0%|          | 0/19 [00:00<?, ?it/s]

all_probs:       <class 'numpy.ndarray'> (150, 6)
all_preds:       <class 'numpy.ndarray'> (150, 6)
all_true_labels: <class 'numpy.ndarray'> (150, 6)
              precision    recall  f1-score   support

         135       1.00      0.20      0.33         5
         136       0.43      0.19      0.26        16
         137       0.57      1.00      0.73        81
         138       0.85      1.00      0.92       128
         139       0.66      1.00      0.80        99
         390       0.00      0.00      0.00         6

   micro avg       0.69      0.93      0.79       335
   macro avg       0.59      0.56      0.51       335
weighted avg       0.69      0.93      0.78       335
 samples avg       0.70      0.95      0.79       335

ROC AUC: 0.9103420948355567


In [33]:
print("First evaluation successfully completed.")

First evaluation successfully completed.


In [34]:
# Second evaluate results; save to /content

eval_output = trainer.evaluate()

# Save evaluate results
with open("eval_results.json", "w") as f:
  json.dump(eval_output, f, indent=4)

In [35]:
print("Second evaluation successfully completed.")

Second evaluation successfully completed.


## Upload model, tokenizer, train results, evaluate results

In [36]:
# Save model to /content

model_path = "model"
trainer.save_model(model_path)

In [37]:
# Upload model and tokenizer to the HF repo_id_model

model     = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

model.push_to_hub(repo_id_model)
tokenizer.push_to_hub(repo_id_model)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/claudelepere/skill_classification/commit/cacfec56c16314f59e738286ece4b6e9050db39c', commit_message='Upload tokenizer', commit_description='', oid='cacfec56c16314f59e738286ece4b6e9050db39c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='model', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

In [38]:
# Upload train_results.json and eval_results.json to the HF repo_id_dataset BETTER to upload to wanddb repo?

upload_file(
    path_or_fileobj = "train_results.json",
    path_in_repo    = "train_results.json",
    repo_id         = HF_name,
    repo_type       = "dataset"
    )

upload_file(
    path_or_fileobj = "eval_results.json",
    path_in_repo    = "eval_results.json",
    repo_id         = HF_name,
    repo_type       = "dataset"
    )

CommitInfo(commit_url='https://huggingface.co/datasets/claudelepere/skill_classification/commit/32d94a9a893ab2833556dce6ff59892fdc098a11', commit_message='Upload eval_results.json with huggingface_hub', commit_description='', oid='32d94a9a893ab2833556dce6ff59892fdc098a11', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

## Test

In [39]:
# Test: first results NO SAVE

get_results(model=model, dataset=test_dataset, batch_size=batch_size, threshold=threshold)

  0%|          | 0/19 [00:00<?, ?it/s]

all_probs:       <class 'numpy.ndarray'> (150, 6)
all_preds:       <class 'numpy.ndarray'> (150, 6)
all_true_labels: <class 'numpy.ndarray'> (150, 6)
              precision    recall  f1-score   support

         135       1.00      0.20      0.33         5
         136       0.20      0.08      0.12        12
         137       0.58      1.00      0.73        84
         138       0.87      1.00      0.93       131
         139       0.67      1.00      0.80       101
         390       0.50      0.12      0.20         8

   micro avg       0.70      0.94      0.80       341
   macro avg       0.64      0.57      0.52       341
weighted avg       0.71      0.94      0.79       341
 samples avg       0.71      0.95      0.80       341

ROC AUC: 0.9140589343139981


In [40]:
print("First test successfully completed.")

First test successfully completed.


In [41]:
# Test: second results NO SAVE

predictions = trainer.predict(test_dataset)

#print(f"predictions.predictions: {type(predictions.predictions)} {predictions.predictions.shape}\n{predictions.predictions}")  # Model logits
#print(f"predictions.label_ids: {type(predictions.label_ids)} {predictions.label_ids.shape}\n{predictions.label_ids}")          # Ground truth labels
print(f"predictions.metrics: {type(predictions.metrics)} {predictions.metrics.shape}\n{predictions.metrics}")                  # Metrics


[[-3.3764715  -2.7061534  -0.20836708  2.739054    1.9581966  -3.7003758 ]
 [-4.089819   -2.7787352   0.62746257  2.972756    1.5420597  -4.0408907 ]
 [-2.6219218  -2.2728655  -1.003358    2.315225    1.8821499  -3.0580378 ]
 [-3.3989518  -2.4041421   1.2457361   2.8897965   1.4013484  -3.6827133 ]
 [-3.2362964  -1.9611675   1.7616155   2.4384735   0.66535556 -3.2235012 ]
 [-3.296511   -2.581292    0.30549374  2.8711243   1.8291818  -3.6532192 ]
 [-2.5497217  -2.2662685  -1.4094818   2.071454    2.078668   -2.8491201 ]
 [-2.244874   -2.0233781  -1.2031306   2.1606429   1.860781   -2.6882317 ]
 [-3.6035259  -2.7836506  -0.50008166  2.5416603   1.9353874  -3.6234326 ]
 [-3.6995256  -2.7840662  -0.17097063  2.8177786   2.000362   -3.8014417 ]
 [-3.3293676  -2.180398    1.2265326   2.0899749   0.10936347 -3.1464915 ]
 [-3.2552097  -2.009903    1.7455698   2.4942012   0.62916887 -3.2586899 ]
 [-3.3894086  -2.5895617  -0.09272584  2.7308087   1.7456754  -3.6739094 ]
 [-3.551312   -2.8808746 

In [42]:
print("Second test successfully completed.")

Second test successfully completed.


### Or otherwise

In [43]:
# Test: third results NO SAVE

predictions = trainer.predict(test_dataset)
print(predictions.predictions)  # Model logits
print(predictions.label_ids)    # Ground truth labels
print(predictions.metrics)      # Metrics

[[-3.3764715  -2.7061534  -0.20836708  2.739054    1.9581966  -3.7003758 ]
 [-4.089819   -2.7787352   0.62746257  2.972756    1.5420597  -4.0408907 ]
 [-2.6219218  -2.2728655  -1.003358    2.315225    1.8821499  -3.0580378 ]
 [-3.3989518  -2.4041421   1.2457361   2.8897965   1.4013484  -3.6827133 ]
 [-3.2362964  -1.9611675   1.7616155   2.4384735   0.66535556 -3.2235012 ]
 [-3.296511   -2.581292    0.30549374  2.8711243   1.8291818  -3.6532192 ]
 [-2.5497217  -2.2662685  -1.4094818   2.071454    2.078668   -2.8491201 ]
 [-2.244874   -2.0233781  -1.2031306   2.1606429   1.860781   -2.6882317 ]
 [-3.6035259  -2.7836506  -0.50008166  2.5416603   1.9353874  -3.6234326 ]
 [-3.6995256  -2.7840662  -0.17097063  2.8177786   2.000362   -3.8014417 ]
 [-3.3293676  -2.180398    1.2265326   2.0899749   0.10936347 -3.1464915 ]
 [-3.2552097  -2.009903    1.7455698   2.4942012   0.62916887 -3.2586899 ]
 [-3.3894086  -2.5895617  -0.09272584  2.7308087   1.7456754  -3.6739094 ]
 [-3.551312   -2.8808746 

In [44]:
print("Third test successfully completed.")

Third test successfully completed.
